In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET
from nltk import word_tokenize
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import pymorphy2
%matplotlib inline

C:\Anaconda3\lib\site-packages\gensim-1.0.1-py3.5-win-amd64.egg\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-07-27 14:09:58,951 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
def tree_df(tree_name):
    tree = ET.parse(tree_name)
    sentences = []
    label = []
    data_item = []
    data = []
    polarity=[]
    root = tree.getroot()
    for sentence in root.iter('sentence'):
        for text in sentence.iter('text'):
            data_item.append(text.text)
        for opinion in sentence.iter('Opinion'):
            label.append(opinion.get('category'))

        label = list(set(label))
        data_item.append(label)
        label=[]
        data.append(data_item)
        data_item=[]
    df = pd.DataFrame(data, columns = ['sentence', 'category'])
    return df

# BAG OF WORDS model

In [3]:
def tokenizator(text):
    sen_clear = []
    for sentence in np.array(text):
        sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
        words = sen_cl.lower().split()
        words_c = []
        for word in words:
            if word != '-':
                words_c.append(morph.parse(word)[0].normal_form)
        sen_clear.append(words_c)
    return sen_clear

In [4]:
def tokenizator_2(sentence):
    sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
    words = sen_cl.lower().split()
    words_c = []
    for word in words:
        if word != '-':
            words_c.append(morph.parse(word)[0].normal_form)
    return words_c

In [5]:
name = 'se16_ru_rest_train.xml'
df = tree_df(name)
test_name = 'se16_ru_rest_test.xml'
df_test = tree_df(test_name)
morph = pymorphy2.MorphAnalyzer()

2017-07-27 14:10:30,255 : INFO : Loading dictionaries from C:\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-07-27 14:10:30,752 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [6]:
df

,sentence,category
0,Добрый час суток.,[]
1,Меня зовут Влада!,[]
2,Я со своим молодым человеком и пару его друзей...,[]
3,Обслуживание хорошее нас встретил метрдотель и...,[SERVICE#GENERAL]
4,"Он был в уютном уголке в конце главного зала, ...",[AMBIENCE#GENERAL]
5,"Просто мы пришли днем часиков в четыре, так чт...",[]
6,На наш не большой банкетик мы заказали пару пр...,"[FOOD#PRICES, FOOD#STYLE_OPTIONS, FOOD#QUALITY]"
7,"Ближе к вечеру нас предупредили о том, что буд...",[AMBIENCE#GENERAL]
8,В общем мы остались безумно довольны проведённ...,[RESTAURANT#GENERAL]
9,Были в Дитае на прошлой неделе с подругой.,[]


In [6]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

In [7]:
train_sentences = tokenizator(df['sentence'])
test_sentences = tokenizator(df_test['sentence'])

In [8]:
%%time
# training
#count_vectorizer = CountVectorizer(
#    analyzer="word", tokenizer=nltk.word_tokenize,
#    preprocessor=None, stop_words=stop_words, max_features=1000) 
count_vectorizer = CountVectorizer(stop_words=stop_words, max_features=1000,analyzer="word")
X_train_bag = count_vectorizer.fit_transform(df.iloc[:,0])
X_test = count_vectorizer.transform(df_test.iloc[:,0])

Wall time: 115 ms


In [9]:
mlb = mlb = MultiLabelBinarizer()
df_all = pd.concat([df, df_test], axis = 0)
y_all = mlb.fit_transform(df_all.iloc[:,1])
y_train = y_all[:len(df)]
y_test = y_all[len(df):]
#clf_bag = OneVsRestClassifier(SVC(kernel='linear', random_state=241, probability=True))
#clf_bag.fit(X_train,y_train)

In [10]:
def classify_SVC(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(SVC(kernel='linear', random_state=241, probability=True))
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    #coverage_error = metrics.coverage_error(y_test, clf.predict_proba(X_test))
    return df_test_f, accuracy, precision, recall, f1


In [11]:
def classify_logreg(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(linear_model.LogisticRegression(n_jobs=1, C=1e5))
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    #coverage_error = metrics.coverage_error(y_test, clf.predict_proba(X_test))
    return df_test_f, accuracy, precision, recall, f1


In [12]:
def classify_NB(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(MultinomialNB())
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    #coverage_error = metrics.coverage_error(y_test, clf.predict_proba(X_test))
    return df_test_f, accuracy, precision, recall, f1

In [13]:
def classify_random_forest(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(RandomForestClassifier( n_estimators = 100 ))
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    #coverage_error = metrics.coverage_error(y_test, clf.predict_proba(X_test))
    return df_test_f, accuracy, precision, recall, f1

In [14]:
df_test_f_bag_SVC, accuracy_bag_SVC, precision_bag_SVC, recall_bag_SVC, f1_bag_SVC = classify_SVC(X_train_bag,y_train,X_test,y_test)

In [15]:
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bag_SVC, precision_bag_SVC, recall_bag_SVC, f1_bag_SVC))

accuracy = 0.56741, precision = 0.58246, recall = 0.37026, f1 = 0.43181


In [16]:
df_test_f_bag_SVC.to_csv(path_or_buf='predictions_BOW_SVC.csv', sep = ',')

In [17]:
df_test_f_bag_logreg, accuracy_bag_logreg, precision_bag_logreg, recall_bag_logreg, f1_bag_logreg = classify_logreg(X_train_bag,y_train,X_test,y_test)

In [18]:
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bag_logreg, precision_bag_logreg, recall_bag_logreg, f1_bag_logreg))

accuracy = 0.38958, precision = 0.30725, recall = 0.35846, f1 = 0.31476


In [19]:
df_test_f_bag_NB, accuracy_bag_NB, precision_bag_NB, recall_bag_NB, f1_bag_NB = classify_NB(X_train_bag,y_train,X_test,y_test)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bag_NB, precision_bag_NB, recall_bag_NB, f1_bag_NB))

accuracy = 0.56989, precision = 0.62622, recall = 0.32283, f1 = 0.36922


In [21]:
df_test_f_bag_forest, accuracy_bag_forest, precision_bag_forest, recall_bag_forest, f1_bag_forest = classify_random_forest(X_train_bag,y_train,X_test,y_test)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bag_forest, precision_bag_forest, recall_bag_forest, f1_bag_forest))

accuracy = 0.52688, precision = 0.52549, recall = 0.29454, f1 = 0.33949


# TF.IDF

In [23]:
tfidfv = TfidfVectorizer(stop_words=stop_words, max_features=1000,analyzer="word")
X_train_tfidf = tfidfv.fit_transform(df.iloc[:,0])
X_test_tfidf = tfidfv.transform(df_test.iloc[:,0])
df_test_f_tfidf_SVC, accuracy_tfidf_SVC, precision_tfidf_SVC, recall_tfidf_SVC, f1_tfidf_SVC = classify_SVC(X_train_tfidf,y_train,X_test_tfidf,y_test)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_SVC, precision_tfidf_SVC, recall_tfidf_SVC, f1_tfidf_SVC))

accuracy = 0.56824, precision = 0.60575, recall = 0.30954, f1 = 0.38708


In [25]:
df_test_f_tfidf_SVC.to_csv(path_or_buf='predictions_tfidf_SVC.csv', sep = ',')

In [26]:
df_test_f_tfidf_logreg, accuracy_tfidf_logreg, precision_tfidf_logreg, recall_tfidf_logreg, f1_tfidf_logreg = classify_logreg(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_logreg, precision_tfidf_logreg, recall_tfidf_logreg, f1_tfidf_logreg))

accuracy = 0.38131, precision = 0.33941, recall = 0.39177, f1 = 0.34375


In [27]:
df_test_f_tfidf_NB, accuracy_tfidf_NB, precision_tfidf_NB, recall_tfidf_NB, f1_tfidf_NB = classify_NB(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_NB, precision_tfidf_NB, recall_tfidf_NB, f1_tfidf_NB))

accuracy = 0.53598, precision = 0.34437, recall = 0.17311, f1 = 0.21846


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
df_test_f_tfidf_forest, accuracy_tfidf_forest, precision_tfidf_forest, recall_forest_NB, f1_tfidf_forest = classify_random_forest(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_forest, precision_tfidf_forest, recall_forest_NB, f1_tfidf_forest))

accuracy = 0.54756, precision = 0.61664, recall = 0.25851, f1 = 0.30811


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Word2Vec

In [29]:
sen_clear = []
for sentence in np.array(df['sentence']):
    sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
    #words = sen_cl.lower().split()
    words = sen_cl.split()
    words_c = []
    for word in words:
        if word != '-':
            words_c.append(word)
    sen_clear.append(words_c)
sen_clear_test = []
for sentence in np.array(df_test['sentence']):
    sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
    #words = sen_cl.lower().split()
    words = sen_cl.split()
    words_c = []
    for word in words:
        if word != '-':
                words_c.append(word)
    words_stop = []
    for word in words_c:
        for stop_word in stop_words:
            if word != stop_word:
                words_stop.append(word)
                break
    sen_clear_test.append(words_c)

In [30]:
model = gensim.models.Word2Vec(sen_clear,size=1000,window=5,min_count=5, workers=4)

2017-06-18 23:24:08,117 : INFO : collecting all words and their counts
2017-06-18 23:24:08,118 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-18 23:24:08,137 : INFO : collected 8579 word types from a corpus of 41494 raw words and 3655 sentences
2017-06-18 23:24:08,138 : INFO : Loading a fresh vocabulary
2017-06-18 23:24:08,152 : INFO : min_count=5 retains 1330 unique words (15% of original 8579, drops 7249)
2017-06-18 23:24:08,153 : INFO : min_count=5 leaves 30900 word corpus (74% of original 41494, drops 10594)
2017-06-18 23:24:08,162 : INFO : deleting the raw counts dictionary of 8579 items
2017-06-18 23:24:08,163 : INFO : sample=0.001 downsamples 56 most-common words
2017-06-18 23:24:08,164 : INFO : downsampling leaves estimated 24335 word corpus (78.8% of prior 30900)
2017-06-18 23:24:08,165 : INFO : estimated required memory for 1330 words and 1000 dimensions: 11305000 bytes
2017-06-18 23:24:08,172 : INFO : resetting layer weights
2017-06-18 23:

In [31]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,))
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0.0:
        featureVec = np.true_divide(featureVec,nwords)
    else:
        featureVec = np.zeros((num_features,))
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features))
    #reviewFeatureVecs = np.array((len(reviews),num_features))
    # 
    # Loop through the reviews
    for i in range(len(reviews)):
        reviewFeatureVecs[i] = makeFeatureVec(reviews[i], model, num_features)
        
    #for review in reviews:

       # Call the function (defined above) that makes average feature vectors
       #reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
       #    num_features)
       #
       # Increment the counter
       #counter = counter + 1.
    
    return reviewFeatureVecs

In [32]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.


trainDataVecs = getAvgFeatureVecs( sen_clear, model, 1000 )

print ("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs( sen_clear_test, model, 1000 )

Creating average feature vecs for test reviews


In [33]:
df_test_f_w2v, accuracy_w2v, precision_w2v, recall_w2v, f1_w2v = classify_random_forest(trainDataVecs,y_train,testDataVecs,y_test)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v, precision_w2v, recall_w2v, f1_w2v))

accuracy = 0.27792, precision = 0.23599, recall = 0.01727, f1 = 0.03111


In [44]:
def classify_SVC2(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(SVC(kernel='linear', random_state=241, probability=True))
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
        cl=[]
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    #coverage_error = metrics.coverage_error(y_test, clf.predict_proba(X_test))
    return df_test_f, accuracy, precision, recall, f1, clf.predict_proba(X_test)


In [45]:
df_test_f_w2v_SVC, accuracy_w2v_SVC, precision_w2v_SVC, recall_w2v_SVC, f1_w2v_SVC, probabilities = classify_SVC2(trainDataVecs,y_train,testDataVecs,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_SVC, precision_w2v_SVC, recall_w2v_SVC, f1_w2v_SVC))

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy = 0.24897, precision = 0.00000, recall = 0.00000, f1 = 0.00000


In [37]:
from sklearn.naive_bayes import GaussianNB
def classify_NB(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(GaussianNB())
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'micro')
    recall = metrics.recall_score(y_test, y_pred, average = 'micro')
    f1=metrics.f1_score(y_test, y_pred, average = 'micro')
    #coverage_error = metrics.coverage_error(y_test, clf.predict_proba(X_test))
    return df_test_f, accuracy, precision, recall, f1

In [38]:
df_test_f_w2v_NB, accuracy_w2v_NB, precision_w2v_NB, recall_w2v_NB, f1_w2v_NB = classify_NB(trainDataVecs,y_train,testDataVecs,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_NB, precision_w2v_NB, recall_w2v_NB, f1_w2v_NB))

accuracy = 0.00000, precision = 0.08800, recall = 0.70779, f1 = 0.15654


In [39]:
df_test_f_w2v_logreg, accuracy_w2v_logreg, precision_w2v_logreg, recall_w2v_logreg, f1_w2v_logreg = classify_logreg(trainDataVecs,y_train,testDataVecs,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_logreg, precision_w2v_logreg, recall_w2v_logreg, f1_w2v_logreg))

accuracy = 0.44500, precision = 0.47043, recall = 0.16055, f1 = 0.23027


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
df_test_f_w2v_NB

,sentence,category,0
0,"Очень милый, уютный ресторанчик со скромными ц...","[AMBIENCE#GENERAL, FOOD#QUALITY, FOOD#STYLE_OP...","[AMBIENCE#GENERAL, DRINKS#PRICES, DRINKS#QUALI..."
1,"Мы отмечали день рожденья,нам разрешили принес...",[],"[AMBIENCE#GENERAL, DRINKS#PRICES, DRINKS#QUALI..."
2,"Салаты со свежайшей зеленью, мясо и курица неж...",[FOOD#QUALITY],"[AMBIENCE#GENERAL, DRINKS#PRICES, DRINKS#QUALI..."
3,Официантки вежливые и улыбчивые.,[SERVICE#GENERAL],"[RESTAURANT#GENERAL, RESTAURANT#MISCELLANEOUS,..."
4,"Единственное,что немного не понравилось - долг...",[SERVICE#GENERAL],"[AMBIENCE#GENERAL, DRINKS#PRICES, DRINKS#QUALI..."
5,Но это мелочи.,[],"[AMBIENCE#GENERAL, DRINKS#QUALITY, DRINKS#STYL..."
6,Общее впечатление прекрасное.,[RESTAURANT#GENERAL],"[RESTAURANT#MISCELLANEOUS, RESTAURANT#PRICES]"
7,Советуем!,[RESTAURANT#GENERAL],"[RESTAURANT#MISCELLANEOUS, RESTAURANT#PRICES]"
8,"Отличный ресторан, была здесь уже не раз и вот...",[RESTAURANT#GENERAL],"[AMBIENCE#GENERAL, FOOD#QUALITY, FOOD#STYLE_OP..."
9,"И я, и все гости остались очень довольны.",[RESTAURANT#GENERAL],"[AMBIENCE#GENERAL, FOOD#QUALITY, FOOD#STYLE_OP..."


In [41]:
probabilities

array([[ 0.5530709 ,  0.00395601,  0.02099473, ...,  0.0088642 ,
         0.02201112,  0.52921756],
       [ 0.2565465 ,  0.00389565,  0.02111161, ...,  0.00891893,
         0.02111357,  0.58832518],
       [ 0.37840878,  0.00368075,  0.02184743, ...,  0.00925152,
         0.02265324,  0.4566768 ],
       ..., 
       [ 0.30730816,  0.00391172,  0.02157755, ...,  0.0089129 ,
         0.02395149,  0.36956458],
       [ 0.36952286,  0.00364612,  0.02888676, ...,  0.00897127,
         0.03177151,  0.44817577],
       [ 0.47244132,  0.00380697,  0.02766226, ...,  0.0092703 ,
         0.0304275 ,  0.5790748 ]])

In [42]:
classes = list(mlb.classes_)
print(classes)

['AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE_OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']


In [43]:
classes_final=[]
for line in probabilities:
    cl = []
    for i in range(len(classes)):
        if line[i] >= 0.2:
            cl.append(classes[i])
    classes_final.append(cl)
    cl=[]
print(classes_final)

[['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['RESTAURANT#GENERAL', 'SERVICE#GENERAL'], [], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'RESTAURANT#GENERAL', 